In [ ]:
import os
import shutil
from PIL import Image
import glob
from pathlib import Path
import split_folders
import warnings
warnings.filterwarnings('ignore')
import numpy as np
import skimage.io as io
from skimage.transform import rotate, AffineTransform, warp, rescale, resize, downscale_local_mean
from skimage.util import random_noise
from skimage.filters import gaussian
import matplotlib.pyplot as plt
from PIL import ImageOps
import torchvision
from torchvision import transforms
import random

In [ ]:
shutil.rmtree("C:/Users/prera/Desktop/Pokemon-Project/Data 2/data_split", ignore_errors=True)
split_folders.ratio('C:/Users/prera/Desktop/Pokemon-Project/Data 2/data_trimmed', output="C:/Users/prera/Desktop/Pokemon-Project/Data 2/data_split", seed=42, ratio=(.8, .2))

Copying files: 8805 files [00:21, 402.63 files/s]


In [ ]:
train_list = []

for root, dirs, files in os.walk("C:/Users/prera/Desktop/Pokemon-Project/Data 2/data_split/train"):
        for file in files:
            if file.endswith(".png"):
                pic_path = os.path.join(root, file).replace("\\","/")
                train_list.append(pic_path)

In [ ]:
def spin(img, ang):
  if ang < -90:
    ang = -90
  if ang > 90:
    ang = 90
  rotated = rotate(img, angle=ang, mode = 'constant')
  return rotated

def translate(img,x,y):
  if (x > 16) or (y > 16):
    transform = AffineTransform(translation=(16,16))
  else:
    transform = AffineTransform(translation=(x,y))
  moved = warp(img,transform,mode='constant')
  return moved

def shift(img,x,y):
  transform = AffineTransform(translation=(x,y))
  wrapShift = warp(img,transform,mode='wrap')
  return wrapShift

def flipHor(img):
  flipLR = np.fliplr(img)
  return flipLR

def flipVer(img):
  flipUD = np.flipud(img)
  return flipUD

def crop(img, c):
  if c >= 32:
    c = 16
  cropped = img[c:(img.shape[0]-c),c:(img.shape[1]-c)]
  scaleFactor = img.shape[0]/(img.shape[0] - 2.0*c)
  resized = resize(cropped, (cropped.shape[0] * scaleFactor, cropped.shape[1] * scaleFactor), anti_aliasing=True)
  return resized

In [ ]:
def expand (img, ang=45, transX=12, transY=12, shiftX=24, shiftY=24, c=10):
  # Translate left
  left = translate(img,transX,0)

  # Translate right
  right = translate(img,-transX,0)
    
  # Translate up
  up = translate(img,0,transY)

  # Translate down
  down = translate(img,0,-transY)
    
  # Translate upper left
  upper_left = translate(img,transX,transY)

  # Translate upper right
  upper_right = translate(img,-transX,transY)
    
  # Translate lower left
  lower_left = translate(img,transX,-transY)

  # Translate lower right
  lower_right = translate(img,-transX,-transY)
    
  return [left, right, up, down, upper_left, upper_right, lower_left, lower_right]

In [ ]:
def graph (img, imgs, h, w):
    # Subplot 1
    fig = plt.figure()
    fig.set_figheight(h)
    fig.set_figwidth(w)
    plotX = 4
    plotY = 1
    
    plt.subplot(plotY,plotX,1)
    plt.imshow(imgs[0])
    
    plt.subplot(plotY,plotX,2)
    plt.imshow(imgs[1])
    
    plt.subplot(plotY,plotX,3)
    plt.imshow(imgs[2])
    
    # Subplot 1
    fig = plt.figure()
    fig.set_figheight(h)
    fig.set_figwidth(w)
    
    plt.subplot(plotY,plotX,1)
    plt.imshow(imgs[3])
    
    plt.subplot(plotY,plotX,2)
    plt.imshow(imgs[4])
    
    plt.subplot(plotY,plotX,3)
    plt.imshow(imgs[5])
    
    # Subplot 2
    fig = plt.figure()
    fig.set_figheight(h)
    fig.set_figwidth(w)
    
    plt.subplot(plotY,plotX,1)
    plt.imshow(imgs[6])
    
    plt.subplot(plotY,plotX,2)
    plt.imshow(imgs[7])
    
    plt.subplot(plotY,plotX,3)
    plt.imshow(img)


In [ ]:
for img in train_list:
    im_obj = io.imread(img)
    h = im_obj.shape[0]
    w = im_obj.shape[1]
    trans = expand(im_obj, transX=12, transY=12)
    #graph(im_obj, trans, h, w)
    #break
    n = 0
    for tran in trans:
        tmp_path = img.replace(".png", " t%d" %n)
        tmp_path += ".png"
        io.imsave(tmp_path, tran)
        n += 1

In [ ]:
bkgrnd_list = []

for root, dirs, files in os.walk("C:/Users/prera/Desktop/Pokemon-Project/Data 2/backgrounds"):
        for file in files:
            if file.endswith(".png"):
                pic_path = os.path.join(root, file).replace("\\","/")
                bkgrnd_list.append(pic_path)

In [ ]:
img_list = []

for root, dirs, files in os.walk("C:/Users/prera/Desktop/Pokemon-Project/Data 2/data_split/"):
        for file in files:
            if file.endswith(".png"):
                pic_path = os.path.join(root, file).replace("\\","/")
                img_list.append(pic_path)

In [ ]:
shutil.rmtree("C:/Users/prera/Desktop/Pokemon-Project/Data 2/data_masked", ignore_errors=True)
Path("C:/Users/prera/Desktop/Pokemon-Project/Data 2/data_masked").mkdir(parents=True, exist_ok=True)
for j in ['train', 'val']:
    for i in range(0,5):
        Path("C:/Users/prera/Desktop/Pokemon-Project/Data 2/data_masked/"+str(j)+"/"+str(i)).mkdir(parents=True, exist_ok=True)

In [ ]:
for img in img_list:
    bkgrnd_path = bkgrnd_list[random.randint(0,len(bkgrnd_list)-1)]
    background = Image.open(bkgrnd_path)
    foreground = Image.open(img)
    background = background.resize((foreground.size[0], foreground.size[1]), resample=Image.LANCZOS)
    background.paste(foreground, (0, 0), foreground.convert('RGBA'))        
    file_path = img.replace("data_split", "data_masked")
    background.save(file_path,"PNG") 